### 1. Загрузка данных

In [10]:
import pandas as pd
data_train = pd.read_csv('../Data/orange_small_churn_data.txt')
data_test = pd.read_csv('../Data/orange_small_churn_test_data.csv')
labels = pd.read_csv('../Data/orange_small_churn_labels.txt', header=None)
data_train['label'] = labels

### 2. Работа с незаполненными значениями

Отбросим признаки, у которых больше половины значений незаполнены - их полезность сомнительна

In [11]:
bad_features = data_train.columns[data_train.isnull().mean(axis=0)>0.5]
data_train = data_train.drop(bad_features, axis=1)
data_test = data_test.drop(bad_features, axis=1)
print data_train.shape

(40000, 70)


Рассмотрим отдельно вещественные и категориальные признаки

In [12]:
all_real_features = ["Var"+str(i) for i in xrange(1, 191)]
real_features = data_train.columns[data_train.columns.isin(all_real_features)]
cat_features = data_train.columns[~data_train.columns.isin(all_real_features+['label'])]

В вещественных признаках заменим пропуски на средние значения признаков и произведем стандартизацию признаков

In [13]:
from sklearn.preprocessing import StandardScaler
means = data_train[real_features].mean(axis=0, skipna=True)
data_train[real_features] = data_train[real_features].fillna(means)
data_test[real_features] = data_test[real_features].fillna(means)
sc = StandardScaler()
data_train[real_features] = sc.fit_transform(data_train[real_features])
data_test[real_features] = sc.transform(data_test[real_features])

В категориальных признаках отбросим те из них, которые имеют слишком много категорий. В качестве порога выбрано 100, при этом получается приемлемое количество признаков. Иначе сильно вероятно переобучение, учитывая размер выборки в 30000.

In [14]:
import numpy as np
num_cat = np.array([np.unique(data_train[feat]).shape[0] for feat in cat_features])
bad_cat_features = cat_features[num_cat>100]
data_train = data_train.drop(bad_cat_features, axis=1)
data_test = data_test.drop(bad_cat_features, axis=1)
cat_features = cat_features[~cat_features.isin(bad_cat_features)]

В категориальных признаках произведем dummy кодирование, при этом пропуск будем расценивать как еще одну категорию

In [15]:
data_train = pd.get_dummies(data_train, dummy_na=True, columns = cat_features, drop_first=True)
data_test = pd.get_dummies(data_test, dummy_na=True, columns = cat_features, drop_first=True)
print data_train.shape
print data_test.shape

(40000, 386)
(10000, 352)


Посольку не все категории у категориальных признаков могут встречаться в обоих датасетах, нужно добавить недостающие признаки с нулевыми значениями.

In [16]:
cols = data_test.columns
for col in cols:
    if not col in data_train.columns:
        data_test = data_test.drop(col, axis=1)
for col in data_train.columns:
    if not col in data_test.columns:
        data_test[col] = [0]*data_test.shape[0]
print data_train.shape
print data_test.shape

(40000, 386)
(10000, 386)


### 3. Обучение моделей и анализ результатов

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score, StratifiedKFold

Оценим значение метрики ROC AUC (она используется в kaggle соревновании) для трех моделей по кросс-валидации с 5 фолдами со стратификацией.

In [18]:
models = [LogisticRegression(), RandomForestClassifier(), GradientBoostingClassifier()]
X = data_train.drop('label', axis=1)
y = data_train.label
scores = []
for model in models:
    score = cross_val_score(model, X, y, scoring='roc_auc', cv=StratifiedKFold(y, n_folds=5))
    scores.append(score)

In [19]:
print "Scores \n Logistic regression: {} \n Random forest: {} \n Gradient boosting classifier: {}".format(*np.array(scores).mean(axis=1))

Scores 
 Logistic regression: 0.639065024784 
 Random forest: 0.573178597222 
 Gradient boosting classifier: 0.715718588661


Оценим то же самое но только по вещественным признакам

In [21]:
models = [LogisticRegression(), RandomForestClassifier(), GradientBoostingClassifier()]
X = data_train[real_features]
y = data_train.label
scores = []
for model in models:
    score = cross_val_score(model, X, y, scoring='roc_auc', cv=StratifiedKFold(y, n_folds=5))
    scores.append(score)

In [22]:
print "Scores \n Logistic regression: {} \n Random forest: {} \n Gradient boosting classifier: {}".format(*np.array(scores).mean(axis=1))

Scores 
 Logistic regression: 0.628801703953 
 Random forest: 0.597401419213 
 Gradient boosting classifier: 0.713371658397


Получили примерно такой же результат. При этом оказалось, что на тестовой выборке kaggle модель с только вещественными признаками дает лучше результат.

Обучим GradientBoostingClassifier на всей выборке и сделаем предсказания для тестовой выборки.

In [24]:
gb = GradientBoostingClassifier()
gb.fit(X, y)
y_pred = gb.predict_proba(data_test[real_features])[:, 1]

Запишем предсказания в файл ответа

In [25]:
df = pd.DataFrame({"Id": range(data_test.shape[0]), 'result': y_pred})
df.to_csv('week4_pred.txt', index=False)